In [1]:
%%html
<style>
div.optional {
    display: block;
    background-color: #d7e2ff;
    border-color: #d7e2ff;
    border-left: 5px solid #d7e2ff;
    padding: 0.5em;
}
div.advanced {
    display: block;
    background-color: #fff4d7;
    border-color: #fff4d7;
    border-left: 5px solid #fff4d7;
    padding: 0.5em;
}
</style>

# ACSE-3 (Numerical Methods) <a class="tocSkip">

## Lecture 9: Partial Differential Equations (PDEs) 2 <a class="tocSkip">
    
### Homework exercises <a class="tocSkip">

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Homework" data-toc-modified-id="Homework-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Homework</a></span><ul class="toc-item"><li><span><a href="#Homework---Comparing-linear-solvers-(Jacobi-vs-Gauss-Seidel)" data-toc-modified-id="Homework---Comparing-linear-solvers-(Jacobi-vs-Gauss-Seidel)-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Homework - Comparing linear solvers (Jacobi vs Gauss-Seidel)</a></span></li><li><span><a href="#Homework---matrix-based-solvers-[$\star\star$]---not-really-any-question-here,-just-read-through-solution-for-interest" data-toc-modified-id="Homework---matrix-based-solvers-[$\star\star$]---not-really-any-question-here,-just-read-through-solution-for-interest-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Homework - matrix based solvers [$\star\star$] - not really any question here, just read through solution for interest</a></span><ul class="toc-item"><li><span><a href="#Imposing-boundary-conditions" data-toc-modified-id="Imposing-boundary-conditions-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Imposing boundary conditions</a></span></li><li><span><a href="#Solution---matrix-based-solvers" data-toc-modified-id="Solution---matrix-based-solvers-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Solution - matrix based solvers</a></span><ul class="toc-item"><li><span><a href="#Some-timings" data-toc-modified-id="Some-timings-1.2.2.1"><span class="toc-item-num">1.2.2.1&nbsp;&nbsp;</span>Some timings</a></span></li></ul></li><li><span><a href="#Solution---Timings-and-errors-(numbers-I-obtained---not-all-run-by-default!)" data-toc-modified-id="Solution---Timings-and-errors-(numbers-I-obtained---not-all-run-by-default!)-1.2.3"><span class="toc-item-num">1.2.3&nbsp;&nbsp;</span>Solution - Timings and errors (numbers I obtained - not all run by default!)</a></span></li></ul></li><li><span><a href="#Homework---A-case-with-a-non-zero-RHS-and-homogeneous-Dirichlet-BCs" data-toc-modified-id="Homework---A-case-with-a-non-zero-RHS-and-homogeneous-Dirichlet-BCs-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Homework - A case with a non-zero RHS and homogeneous Dirichlet BCs</a></span></li><li><span><a href="#Homework---A-case-with-a-non-zero-RHS-and-inhomogeneous-Dirichlet-BCs" data-toc-modified-id="Homework---A-case-with-a-non-zero-RHS-and-inhomogeneous-Dirichlet-BCs-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Homework - A case with a non-zero RHS and inhomogeneous Dirichlet BCs</a></span></li><li><span><a href="#Homework---Alternative-solver-methods-for-Navier-Stokes-[$\star\star\star$]" data-toc-modified-id="Homework---Alternative-solver-methods-for-Navier-Stokes-[$\star\star\star$]-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Homework - Alternative solver methods for Navier-Stokes [$\star\star\star$]</a></span></li></ul></li></ul></div>

In [ ]:
%precision 3
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg as sl
import scipy.sparse as sp
import scipy.sparse.linalg as spla
# the following allows us to plot triangles indicating convergence order
from mpltools import annotation
# as we're in 2D we will be doing some 3D plotting
from mpl_toolkits.mplot3d import Axes3D
# and using some colormaps
from matplotlib import cm
# and we will create some animations!
import matplotlib.animation as animation
from IPython.display import HTML
from pprint import pprint

# Homework


## Homework - Comparing linear solvers (Jacobi vs Gauss-Seidel)

In the lecture we considered the problem 

$$ \frac{\partial^2 c}{\partial x^2} + \frac{\partial^2 c}{\partial y^2} \equiv \nabla^2 c = f, $$

with an analytical solution which was chosen such that its Laplacian is zero (i.e. $f\equiv 0$)

$$ c_{\text{exact}}(x,y) =\frac{\sin(2\pi x)\sinh(2\pi y)}{\sinh(2\pi)},$$

and we used the value of this exact solution to define Dirichlet BCs for our problem.

The difference between the Jacobi solver we used in the lecture and Gauss-Seidel is that with G-S we make use of updated solution values as soon as they are available - we don't wait until the next iteration to make use of them. We saw in L3 that this approach can lead to faster convergence (in terms of the number of  required iterations - of course not overall if each iteration costs a different amount).

We can write the iterative scheme in the Gauss-Seidel case as:

$$ c^{k+1}_{i,j} = \frac{1}{4}\left(c^{k}_{i+1,j}  + c_{i-1,j}^{k+1} + c^{k}_{i,j+1} + c^{k+1}_{i,j-1} - \Delta x^2 f_{i,j} \right), $$

where we have made an assumption on the order we perform the update in $i$ and $j$, i.e. for a given $i,j$ we assume we have already visiting and updated the values at the $i-1$ and the $j-1$ locations.

Write some code to compare the number of iterations and overall time to solve the above Poisson problem with Dirichlet BCs  to a given tolerance using our vectorised Jacobi code from the lecture, as well as non-vectorised versions of Jacobi and Gauss-Seidel. 

In order to compute timings you could turn these three solvers into functions and use `%timeit` as we have done in previous homeworks.

<div class="advanced">

## Homework - matrix based solvers [$\star\star$] - not really any question here, just read through solution for interest

    
***The following is taken direct from the lecture - it then extends by considering some matrix based solver options***
    
<br>

Alternatively (to open up a wider range of solver options) we can cast the problem is the matrix form

$$A\boldsymbol{C} = \boldsymbol{b},$$

and solve with standard methods such as Gaussian elimination or conjugate gradients (we will see that for the Laplace operator $A$ is symmetric positive definite).

The first step for constructing the matrix-vector multiplication on the LHS is to assume a numbering which allows us to reshape the $N_x\times N_y$ unknowns making up 

$$\boldsymbol{c} = \{c_{ij}\}, \;\;\;\;\; i=0,\ldots, N_x-1,\;\;\;\; j=0,\ldots, N_y-1,$$  

into an $N_xN_y \times 1$ column vector 

$$\boldsymbol{C} = \{C_{k}\}, \;\;\;\;\; k=0,\ldots, N_x\times N_y-1.$$

There are several ways we can go back and forth between these numberings in Python - read the docs on `reshape`, `flatten`, `ravel`:

(note that these implement what is called [*row major ordering*](https://en.wikipedia.org/wiki/Row-_and_column-major_order))

In [ ]:
a = np.array([[1,2,3],[4,5,6]])
pprint(a)
pprint(a.ravel())
pprint(a.flatten())
pprint(a.reshape(-1))

print('\n')
# this is how we can go back again
a2 = a.ravel()
pprint(a2)
pprint(a2.reshape(np.shape(a)))

print('\n')
# this shows that "C ordering" is the default row-major behaviour we will use
pprint(a.ravel(order='C'))
# and this shows that "Fortran ordering" is column-major
pprint(a.ravel(order='F'))

### Imposing boundary conditions


1. The simplest approach to applying Dirichlet BCs would be to replace the corresponding row of the matrix with a 1 on the diagonal, zero in all other row entries and set the Dirichlet BC value in the corresponding row of the RHS matrix. The problem with this approach is that it destroys the symmetry of the LHS matrix which we can exploit in solvers (and indeed which is a requirement for the use of certain solvers) - this is because we have edited the row, but can't similarly edit the corresponding column without messing up the discretisation for other nodes.


2. The second approach is to leave the row unchanged but set a very large number on the diagonal, and set this large number multiplied by our Dirichlet BC value in the corresponding row of the RHS matrix. This is sometimes called the *big spring* method.


3. And finally *Lifting*, which we do below, which involves removing the rows and columns corresponding to the boundary nodes from the discretisation matrix.

### Solution - matrix based solvers

The code below that assembles the discretisation matrix is quite advanced.

To try and understand what is going on you will need to read what some new NumPy commands do: [`numpy.ix_`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.ix_.html) and [`numpy.kron`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.kron.html)

Note that we use the "lifting" approach to apply BCs - this means we need to remove the rows and columns that correspond to degrees of freedom on the boundary from the solve - we make the matrix problem solver. We will add back in the BC values before ultimately plotting the solution.

In [ ]:
def Poisson_2D_Dirichlet_FD(X, Y, RHS_f, DBC):
    """ Form (i.e. assemble) the discretised Poisson problem 
    in 2D using second-order finite differences,
    with the RHS function given by RHS_F 
    and Dirichlet boundary conditions given in the function DBC.
    
    As we assume Dirichlet BCs in this implementation, here we return 
    the "interior" mesh data structures X_int, i_int etc, and the 
    associated discretised A matrix and RHS vector.

    We don't actually solve the assembled system in this function.
    """
    Nx = np.shape(X)[0]
    Ny = np.shape(Y)[1]
    # our code below is only currently for the case Nx=Ny (and dx=dy)
    assert Nx==Ny
    
    # ID the boundary nodes - initialise the ID to 1 for ALL nodes
    bndry_node = np.ones_like(X)
    # and overwrite with zeros for the interior nodes, leaving 1 only on boundary 
    bndry_node[1:-1, 1:-1] = 0

    # we won't be solving for boundary node values ("lifting approach)", 
    # so define data strucutres for the domain "interior"
    i_int = np.arange(1, Nx-1)
    j_int = np.arange(1, Ny-1)

    # the mesh in the interior - read the docs to see what `numpy.ix_` does:
    # " Using ix_ one can quickly construct index arrays ... 
    # a[np.ix_([1,3],[2,5])] returns the array [[a[1,2] a[1,5]], [a[3,2] a[3,5]]]. "
    X_int = X[np.ix_(i_int, j_int)]
    Y_int = Y[np.ix_(i_int, j_int)]

    # form the LHS discretisation matrix which is built up from some
    # fundamental matrices in x and y
    ex = np.ones((Nx-2, 1))
    ey = np.ones((Ny-2, 1))

    Tx = np.diagflat(ex[1:], k=-1) - 2*np.diagflat(ex, k=0) + np.diagflat(ex[1:], k=1)
    Ty = np.diagflat(ey[1:], k=-1) - 2*np.diagflat(ey, k=0) + np.diagflat(ey[1:], k=1)

    # this is some magic to arrive at the desired discretisation matrix
    # could equally well assemble the discretisation matrix via the appropriate loops
    A_int = (np.kron(np.eye(Ny-2), Tx) + np.kron(Ty, np.eye(Nx-2))) / dx**2
       
    # Use the given function f to set the RHS
    RHS_int = RHS_f(X_int,Y_int)

    # and update the RHS based on given Dirichlet BCs ("lifting approach")
    RHS_int[:, 0] = RHS_int[:, 0] - DBC(X[i_int, 0], Y[i_int, 0]) / dx**2
    RHS_int[:, -1] = RHS_int[:, -1] - DBC(X[i_int, -1], Y[i_int, -1]) / dx**2
    RHS_int[0, :] = RHS_int[0, :] - DBC(X[0, j_int], Y[0, j_int]) / dx**2
    RHS_int[-1, :] = RHS_int[-1, :] - DBC(X[-1, j_int], Y[-1, j_int]) /dx**2   
   
    # row-major numbering
    RHS_int = RHS_int.flatten('C')
    
    return A_int, RHS_int, X_int, Y_int, i_int, j_int

# we'll assume uniform mesh in x and y direction for simplicity
# can easily be generalised

# Size of rectangular domain
Lx = 1
Ly = Lx

# Number of grid points in each direction, including boundary nodes
Nx = 20
Ny = Nx

# hence the mesh spacing
dx = Lx/(Nx-1)
dy = Ly/(Ny-1)

X, Y = np.mgrid[0:Nx:1, 0:Ny:1]
X = dx*X
Y = dy*Y

# Give the functions defining our problem

# If we know the exact solution we can compute exact errors
def c_exact(X,Y):
    return np.sin(2*np.pi*X)*np.sinh(2*np.pi*Y)/np.sinh(2*np.pi)

# A function giving the RHS
def RHS_f(X,Y):
    return np.zeros_like(X)

# A function to return Dirichlet BCs - if we have an exact solution we can 
# just point at that.
def DBC(X,Y):
    return c_exact(X,Y)

# Let's call the above to assemble the discrete system
A_int, RHS_int, X_int, Y_int, i_int, j_int = Poisson_2D_Dirichlet_FD(X,Y,RHS_f,DBC)

In [ ]:
# let's plot the sparsity pattern of our discretisation matrix A

fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(111)
ax.spy(A_int);

In [ ]:
# solve and plot solution

# solve using scipy's solve (a direct method)
C_int = sl.solve(A_int, RHS_int)

# revert back to a 2D numbering format
c_int = C_int.reshape(np.shape(X_int))

# Initialise the full solution over the entire domain via a copy of the exact solution
# this will have the effect of correctly including the Dirichlet BCs in what we 
# ultimately plot
c = np.copy(c_exact(X,Y))

# now overwrite all but the BC values with our interior numerical solution
c[np.ix_(i_int, j_int)] = c_int

# plot the solution as a 3D surface
fig = plt.figure(figsize=(12, 6))

ax = fig.add_subplot(1, 2, 1, projection='3d')
surf = ax.plot_surface(X, Y, c, rstride=1, cstride=1,
                       cmap=cm.coolwarm, linewidth=0, antialiased=True)
fig.colorbar(surf, shrink=0.5, aspect=5)
ax.set_xlim(0, Lx)
ax.set_ylim(0, Ly)
ax.set_xlabel('$x$', fontsize=14)
ax.set_ylabel('$y$', fontsize=14)
ax.set_zlabel('$C$', fontsize=14)
ax.set_title('2D Poisson problem', fontsize=14)

# plot the solution as a 2D contour plot
ax = fig.add_subplot(1, 2, 2)
ax.contour(X, Y, c, cmap=cm.coolwarm)
ax.set_xlim(0, Lx)
ax.set_ylim(0, Ly)
ax.set_xlabel('$x$', fontsize=14)
ax.set_ylabel('$y$', fontsize=14)
ax.set_title('2D Poisson problem', fontsize=14)

print('Error (RMS) = %.4e' % (sl.norm(c_exact(X,Y) - c,  'fro')/(np.sqrt(Nx*Ny))))
print('Error (inf) = %.4e' % (np.max(np.fabs(c_exact(X,Y) - c))))

#### Some timings

Note that this is a situation where we can start generating some quite large matrix systems we may want to solve in anger. 

We have multiple solvers available to us and so it is interesting to compare their computational efficiency

In [ ]:
%%timeit
C_int = sl.solve(A_int, RHS_int)

In [ ]:
%%timeit
# Use CG method
C_int = spla.cg(A_int, RHS_int, x0=None, tol=1e-10, maxiter=1000)

In [ ]:
# As most entries of A are zero let's use a sparse data storage method
sA_int = sp.csc_matrix(A_int)

In [ ]:
%%timeit
# Use CG method on sparse matrix
C_int = spla.cg(sA_int, RHS_int, x0=None, tol=1e-10, maxiter=1000)

### Solution - Timings and errors (numbers I obtained - not all run by default!)


|              |  10 x 10       |  20 x 20       |  40 x 40         |  80 x 80       | 160 x 160      | 
| ------------ |----------------|----------------|------------------|----------------|----------------|
| Error1       | 4.9666e-03     | 1.2096e-03     | 2.9629e-04       | 7.3227e-05     | 1.8197e-05     |
| Error2       | 1.3400e-02     | 3.3101e-03     | 7.9268e-04       | 1.9362e-04     | 4.7849e-05     |
| LU           | 137 µs         | 2.15 ms        | 90.7 ms          |  4.29 s        | 3min 40s       |
| CG           | 523 µs         | 1.45 ms        | 48.5 ms          |  1.56 s        | 49.2s          |
| CG/Sparse    | 502 µs         | 1.03 ms        | 3.43 ms          |  18.6 ms       | 81 ms (231)    | 



In [ ]:
# let's plot the timings

fig = plt.figure(figsize=(12, 6))
ax = fig.add_subplot(121)

DoFs = np.array([10*10, 20*20, 40*40, 80*80, 160*160])

ax.loglog(DoFs, [137e-6, 2.15e-3, 90.7e-3, 4.29, 3*60+40], 'ko-', label='LU')
ax.loglog(DoFs, np.power(3.0e-4*DoFs, 3), 'k--')
ax.loglog(DoFs, np.power(2.5e-4*DoFs, 2.5), 'k--')

ax.loglog(DoFs, [523e-6, 1.45e-3, 48.5e-3, 1.56, 49.2], 'bo-', label='CG')
ax.loglog(DoFs, np.power(1.5e-4*DoFs, 2.5), 'b--')

ax.loglog(DoFs, [502e-6, 1.03e-3, 3.43e-3, 18.6e-3, 81e-3], 'ro-', label='CG - sparse')
ax.loglog(DoFs, 2e-7*DoFs*np.log(DoFs), 'r--')

ax.legend(loc='best')
ax.set_xlabel('Degrees of freedom', fontsize=14)
ax.set_ylabel('Run time [s]', fontsize=14)

ax = fig.add_subplot(122)
DoFs = np.array([10*10, 20*20, 40*40, 80*80, 160*160])
Error1 = np.array([4.9666e-03, 1.2096e-03, 2.9629e-04, 7.3227e-05, 1.8197e-05])
Error2 = np.array([1.3400e-02, 3.3101e-03, 7.9268e-04 ,1.9362e-04, 4.7849e-05])
ax.loglog(DoFs, Error1,'ko-', label='error1')
ax.loglog(DoFs, Error2,'bo-', label='error2')
ax.legend(loc='best')
ax.set_xlabel('Degrees of freedom', fontsize=14)
ax.set_ylabel('Error', fontsize=14);

So the errors are equivalent, and can be seen the converge linearly with total number of degrees of freedom, but since we're in 2D this is equivalent to converging quadratically with $\Delta x$ (or $\Delta y$).

The dashed lines on the left are: 

- black - I've plotted DoFs to the power 2.5 and 3, and our LU results are somewhere in between

- blue - I've plotted  DoFs to the power 2.5 and CG seems to follow this quite well

- red - for the sparse version of CG our results seem to follow DoF*log(DoF)


So notice that exploiting the sparsity of the matrix is vitally important for overall algorithm efficiency.

## Homework - A case with a non-zero RHS and homogeneous Dirichlet BCs

Let's consider now a Poisson problem with a non-zero RHS and zero Dirichlet BCs.

Our PDEs is
$$ \frac{\partial^2 c}{\partial x^2} + \frac{\partial^2 c}{\partial y^2} \equiv \nabla^2 c = f, $$

with exact solution that is constructed to take the value zero on all boundaries:

$$ c_{\text{exact}}(x,y) = x^2 (1 - x)\sin(\pi y). $$

We can easily evaluate the Laplacian of this function to give us the RHS function:

$$ f = \sin(\pi y)  ( 2 - 6 x - (\pi x)^2  (1 - x) ). $$

Solve this problem using one of your solvers from above.

## Homework - A case with a non-zero RHS and inhomogeneous Dirichlet BCs

Consider now a case with

$$ c_{\text{exact}}(x,y) = \exp\left(x + \frac{y}{2}\right), $$

which is non-zero on the boundaries, and hence this same function can be used to specify Dirichlet BCS.

We can easily evaluate the Laplacian of this function to give us the RHS function:

$$ f = \frac{5}{4}\exp\left(x + \frac{y}{2}\right). $$

Solve this problem using one of your solvers.

<div class="advanced">

## Homework - Alternative solver methods for Navier-Stokes [$\star\star\star$]

In the lecture a major cost of our Navier-Stokes solver is the solution of the pressure Poisson equation within the pressure projection method.

Try updating our CFD solver from lecture to use an LU based matrix solver for pressure instead.

You will need to consider several things here:


1. The imposition of homogeneous Neumann BCs in the generation of the discretisation matrix $A$.


2. The imposition of the pressure level node in the generation of the discretisation matrix $A$ (I suggest for simplicity you use the big spring method here).

<br>

**Important comments on performance optimisation** 

Now while simple our Jacobi solver from the lecture actually performs quite well for our lid driven cavity case. Firstly, as considered above, as the solve is fully vectorised that means that each iteration is very efficient. Secondly, as we are solving this problem to steady state we need to do a lot of time step where things don't change much from one to the next. This is an advantage for the iterative solvers utilised within each time step: starting the Jacobi iteration using an initial guess from the previous time level means that we can minimise the number of iterations required to converge. Indeed for the lid-driven cavity we observe that while for early time levels we have to use a large number of iterations, for much of latter parts of the solve Jacobi only requires a single iteration to converge, this is because the pressure is not changing by very much between time levels.  For a fully time-dependent problem a simple solver like Jacobi is not viable, the ability to start from a reasonable initial guess (from the previous time level) and simply the overall size of the problem, points to an optimal solution being a matrix based iterative solver such a preconditioned CG.
    
</div>